In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import StratifiedKFold



In [2]:
def kfold(df = None, time_stamp = 20):
    for label in [0, 1, 2, 3, 4]:
        df_label = df[df['label'] == label]
        
        for i in range(len(df_label)//time_stamp):
            if i == len(df_label)//time_stamp - 1:
                df_stamp = df_label[i*time_stamp:]
            else:
                df_stamp = df_label[i*time_stamp: i*time_stamp + 20]
            
            skf = StratifiedKFold(n_splits=5, shuffle=True)
            X = df_stamp['image_id']
            Y = df_stamp['label']
            fold = 0
            for train_index, test_index in skf.split(X, Y):
                fold += 1
                # print(test_index)
                X_test = X.iloc[test_index]
                df.loc[df.image_id.isin(X_test), 'fold'] = fold
#                 print(X_test)
#                 print(df.image_id.isin(X_test))
    return df 

In [3]:
df_new = pd.read_csv('new.csv')
X = df_new['image_id']
Y = df_new['label']
df_new['fold'] = 0

df_new = df_new.sort_values(by=['image_id'])

In [4]:
df_new = kfold(df_new)

In [5]:
df_new[df_new['label'] == 0]['fold'][:].value_counts()

1    218
5    217
4    217
3    217
2    217
Name: fold, dtype: int64

In [6]:
# df = df_new
# label = 0
# time_stamp = 20
# df_label = df[df['label'] == label]
# len(df_label)//time_stamp

In [7]:
# df_label[0:20]

In [8]:
df_old = pd.read_csv('old.csv')
X = df_old['image_id']
Y = df_old['label']
df_old['fold'] = 0

df_old = df_old.sort_values(by=['image_id'])

In [9]:
df_old = kfold(df_old)

In [10]:
frames = [df_new, df_old]
df_all = pd.concat(frames)
df_all.to_csv('new_mix_v3.csv', index = False)

In [17]:
# df = pd.read_csv('new_mix_v3.csv')
df = pd.read_csv('/home/hana/sonnh/kaggle-cassava/dataset/train_mix/new_mix_1234.csv')

In [23]:
df[df['label'] == 4]['fold'].value_counts()

5    579
3    578
4    578
1    577
2    577
Name: fold, dtype: int64